In [1]:
from pyspark.sql import SparkSession, functions as f

In [2]:
spark = (
    SparkSession.builder.appName("Preprocessing_Yellow")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '3g')   
    .config('spark.executor.memory', '4g')  
    .config('spark.executor.instances', '2')  
    .config('spark.executor.cores', '2')
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
23/09/29 22:12:38 WARN Utils: Your hostname, LAPTOP-RELH58H1 resolves to a loopback address: 127.0.1.1; using 172.19.22.4 instead (on interface eth0)
23/09/29 22:12:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/29 22:12:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/29 22:12:50 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
consumers = spark.read.parquet("../../../data/insights/pre_insights/consumers.parquet/")

In [18]:
consumers_aggregated = consumers.groupBy("merchant_abn").agg(
                        f.countDistinct("consumer_id").alias("number_of_unique_consumers"),
                        f.avg("consumer_fraud_probability_%").alias("average_consumer_fraud_probability"))

In [6]:
consumers_aggregated.filter(consumers_aggregated["number_of_unique_consumers"] == 0 ).show()

+------------+--------------------------+----------------------------------+
|merchant_abn|number_of_unique_consumers|average_consumer_fraud_probability|
+------------+--------------------------+----------------------------------+
+------------+--------------------------+----------------------------------+



In [6]:
repeat_consumers = consumers.groupBy("merchant_abn", "consumer_id").agg(f.count("consumer_id").alias("consumer_order_times"))
repeat_consumers_order_times = repeat_consumers.groupBy("merchant_abn").agg(f.avg("consumer_order_times").alias("average_repeat_transactions_per_consumer"))
repeat_consumers = repeat_consumers.filter(repeat_consumers.consumer_order_times > 1)
repeat_consumers_count = repeat_consumers.groupBy("merchant_abn").agg(f.count("consumer_order_times").alias("number_of_repeat_consumers"))

In [7]:
repeat_consumers_order_times.orderBy(f.col("average_repeat_transactions_per_consumer").desc()).show()

+------------+----------------------------------------+
|merchant_abn|average_repeat_transactions_per_consumer|
+------------+----------------------------------------+
| 24852446429|                      12.008899274137416|
| 86578477987|                      11.325478498632862|
| 64203420245|                      10.818791946308725|
| 49891706470|                      10.287674638293641|
| 46804135891|                        9.72962760403719|
| 45629217853|                       9.485508327119065|
| 89726005175|                       8.960375857611615|
| 43186523025|                       8.348582794629538|
| 80324045558|                        8.17741534483616|
| 63290521567|                       7.564222465426326|
| 68216911708|                       7.516937770482017|
| 21439773999|                       5.043445617898794|
| 64403598239|                       4.755238095238095|
| 72472909171|                       4.294793609193085|
| 94493496784|                        4.19430392

In [8]:
consumers_aggregated = consumers_aggregated.join(repeat_consumers_count, on = "merchant_abn", how = "inner")
consumers_aggregated = consumers_aggregated.join(repeat_consumers_order_times, on = "merchant_abn", how = "inner")

In [9]:
consumers_aggregated.orderBy(f.col("number_of_unique_consumers").asc()).show()

In [10]:
consumers_aggregated.orderBy(f.col("number_of_unique_consumers").desc()).show()

+------------+--------------------------+----------------------------------+--------------------------+----------------------------------------+----------------------+
|merchant_abn|number_of_unique_consumers|average_consumer_fraud_probability|number_of_repeat_consumers|average_repeat_transactions_per_consumer|consumer_retainability|
+------------+--------------------------+----------------------------------+--------------------------+----------------------------------------+----------------------+
| 45629217853|                     20115|              0.042364532680646334|                     20095|                       9.485508327119065|    0.9990057171265225|
| 86578477987|                     20115|              0.043976018822537884|                     20113|                      11.325478498632862|    0.9999005717126522|
| 64203420245|                     20115|               0.04343806405661244|                     20112|                      10.818791946308725|    0.9998508575

In [11]:
consumers_aggregated.write.mode("overwrite").parquet("../../../data/insights/agg_insight_data/consumers_agg.parquet")

In [12]:
spark.stop()